In [1]:
import ee
import geemap as emap

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWj8cuPttKEqdiv7n8DeyMH9OsjKQhkk6OFDQOxWFt_lMdP7bLjkAUk

Successfully saved authorization token.


In [3]:
Map=emap.Map()

# Landsat 8 and test area

In [4]:
# Define the test area Hanoi.
hn=ee.FeatureCollection("users/havantuyen/HN_Mos")
# Load a cloudy Landsat 8 image.
ls8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterDate("2020-01-01","2021-01-01").\
filterBounds(hn).filter(ee.Filter.lt("CLOUD_COVER",30))
# print out the number of images
ls8.size().getInfo()

14

**1. How does `qualityMosaic` function pick up the greenest image or pixel in the image collection?**

In [5]:
# user defined function to generate NDVI 
def ndvi(img):
    NDVI=img.expression("(NIR-RED)/(NIR+RED)",{
        "NIR":img.select("B5"),
    "RED":img.select("B4")}).rename("NDVI")
    day=ee.Image(ee.Number.parse(img.date().format("D"))).rename("DOY").toInt()
    return img.addBands(NDVI).addBands(day)
ls8NDVI=ls8.map(ndvi)

In [6]:
# Using qualityMosaic to find the greenest days of the year
greenest=ls8NDVI.qualityMosaic("NDVI")
Map.centerObject(hn)
Map.addLayer(greenest.select("NDVI"),{"palette":["red","yellow","green"],"max":0.5,"min":0},"Greenest NDVI")
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

**2. How does `ee.Image().toArray()` and `arrayFlatten()` work? is it the same numpy array in Python?**

When should we convert image/images to array?

In [7]:
image=ls8.first().toArray()

**3. How to convert an image dictionary to an Image?**

In [8]:
mdict=ls8.getInfo()["features"]
mdict[0]==ls8.first().getInfo()

True

In [9]:
newImage=ee.Image(ee.Dictionary(mdict[0]))
# Map.addLayer(newImage,{},"Image from Dictionary")
# Map

**4. How did the script below work? and how is the difference between two methods?**

In [11]:
image = ee.Image('COPERNICUS/S2/20190703T050701_20190703T052312_T43PGP')
vis={"min":0,"max":3000,"bands":["B4","B3","B2"]}
Map.centerObject(image)
Map.addLayer(image,vis,"Image")

- Using `bitwiseAnd()` 

In [12]:
# Select QA band
img=image.select("QA60")
# A binary image with 0 indicating cirrus and 1 otherwise
cirrus=img.bitwiseAnd(1<<11).eq(0)
# A binary image with 0 indicating cloud and 1 otherwise
cloud=img.bitwiseAnd(1<<10).eq(0)
# Combine to get an image where 1 indicates no cloud and 0 otherwise
combine=cloud.And(cirrus)

Map.addLayer(combine,{},"Combine")
Map

Map(bottom=61376.0, center=[12.157960082986381, 77.3428094074448], controls=(WidgetControl(options=['position'…

- Without `bitwiseAnd`

In [13]:
# Select QA band
img=image.select("QA60")
# A binary image with 0 indicating cirrus and 1 otherwise
# cirrus=img.eq(0)
# A binary image with 0 indicating cloud and 1 otherwise
cloud=img.eq(0)
# Combine to get an image where 1 indicates no cloud and 0 otherwise
# combine=cloud.And(cirrus)

Map.addLayer(cloud,{},"Without Bitwise") # I have noitced no differences between the two
Map

Map(bottom=61376.0, center=[12.157960082986381, 77.3428094074448], controls=(WidgetControl(options=['position'…

In [14]:
noCloud=image.updateMask(cloud)

**5. How does `ee.Reducer.histogram()` and `ee.Reducer.linearFit()` work? and How to get basic statistics of an image such as `mean`, `max`, etc?**

`.reduce(ee.Reducer.mean()` can be applied for image collection, feature collection or list.

In [50]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

In [51]:
# Is the coordinate below bounding box of the image
image.geometry().getInfo()

{'type': 'Polygon',
 'coordinates': [[[-121.3637119499993, 36.41016684133052],
   [-121.35905784815819, 36.42528989660049],
   [-121.2315833015866, 36.840374852891664],
   [-121.09978718573184, 37.26438246506325],
   [-121.00571062336425, 37.564795515259384],
   [-120.98453376062118, 37.632161601008896],
   [-120.95100979452299, 37.73864548098522],
   [-120.90277241165228, 37.89149086576169],
   [-120.8836409072059, 37.951976016520376],
   [-120.85713152433351, 38.03584247073611],
   [-120.82804345546616, 38.12789513604401],
   [-122.38148159443172, 38.42337450676813],
   [-122.9500220192271, 38.525813632077686],
   [-122.95103687833704, 38.52422133103557],
   [-122.9569591344694, 38.504384836247866],
   [-123.43853932998316, 36.805122381748035],
   [-123.18722447462653, 36.759167415189125],
   [-121.5105534682754, 36.43765126135182],
   [-121.36447385999617, 36.408418528930035],
   [-121.3637119499993, 36.41016684133052]]]}

In [53]:
# Compute min, max or other statistics for each band 
# stats = image.reduceRegion({
#   "reducer": ee.Reducer.mean(),
#   "geometry": image.geometry(),
#   "scale": 100,
#   "maxPixels": 1e10
#   })

**6. How to calculate `mean` from all images or ImageCollection within a certain percentile range?**

In [76]:
ls8=ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterDate("2020-01-01","2020-12-10").filterBounds(hn)

In [81]:
pc10=ls8.reduce(ee.Reducer.percentile([20,50,95]))

In [101]:
# example to get mean or median from certain percentile range
import numpy as np

a=np.array([1,2,3,3,4,6,8,9,445,5,6,-1101010])

def percentileMean(array, lower=25,upper=95):
    lower1=np.percentile(array,lower)
    upper1=np.percentile(array,upper)
    mlist=[]
    for i in array:
        if lower1<=i<=upper1:
            mlist.append(i)
    return np.mean(np.array(mlist)), np.median(np.array(mlist))
percentileMean(a,10,99)

(5.111111111111111, 5.0)

- **How to convert an entire image of a constant to a vector shapefile?**

For exmaple, calculate the number of images over time and location and convert them into vector shapefile, and each shapefile of image represents a number which indicated number of images.

In [23]:
def count(img):
    image=img.select("B2").multiply(0).add(1).rename("quantity").toInt()
    return img.addBands(image)

In [24]:
newls=ls8.map(count)

In [25]:
Map.centerObject(hn)
Map.addLayer(newls.select("quantity").sum(),{"min":1,"max":10,"palette":["green","red"]},"Number")
Map

Map(bottom=14815.0, center=[20.107523268824004, 107.27050781250001], controls=(WidgetControl(options=['positio…

In [22]:
b2=ls8.first().select("B2").multiply(0).add(1)